In [2]:
import argparse
import copy
import json
import os
import sys
import numpy as np
import torch
import yaml
from det3d import torchie
from det3d.datasets import build_dataloader, build_dataset
from det3d.models import build_detector
from det3d.torchie import Config
from det3d.torchie.apis import (
    batch_processor,
    build_optimizer,
    get_root_logger,
    init_dist,
    set_random_seed,
    train_detector,
)
from det3d.torchie.trainer import get_dist_info, load_checkpoint
from det3d.torchie.trainer.utils import all_gather, synchronize
from torch.nn.parallel import DistributedDataParallel
import pickle 
import time 

In [3]:
def save_pred(pred, root):
    with open(os.path.join(root, "prediction.pkl"), "wb") as f:
        pickle.dump(pred, f)


In [4]:
CFG_NAME='waymo_centerpoint_voxelnet_3x'


def parse_args():
    parser = argparse.ArgumentParser(description="Train a detector")
    parser.add_argument("--config", 
                        default=f'./tools/dist_test.py configs/waymo/voxelnet/two_stage/{CFG_NAME}.py', help="train config file path")
    parser.add_argument("--work_dir", 
                        default=f'work_dirs/{CFG_NAME}',
                        # required=True, 
                        help="the dir to save logs and models")
    parser.add_argument(
        "--checkpoint", 
        default=f'work_dirs/{CFG_NAME}/latest.pth',
        help="the dir to checkpoint which the model read from"
    )
    parser.add_argument(
        "--txt_result",
        type=bool,
        default=False,
        help="whether to save results to standard KITTI format of txt type",
    )
    parser.add_argument(
        "--gpus",
        type=int,
        default=1,
        help="number of gpus to use " "(only applicable to non-distributed training)",
    )
    parser.add_argument(
        "--launcher",
        choices=["none", "pytorch", "slurm", "mpi"],
        default="none",
        help="job launcher",
    )
    parser.add_argument("--speed_test", action="store_true")
    parser.add_argument("--local_rank", type=int, default=0)
    parser.add_argument("--testset", action="store_true")

    args = parser.parse_args()
    if "LOCAL_RANK" not in os.environ:
        os.environ["LOCAL_RANK"] = str(args.local_rank)

    return args


In [5]:
import sys

# Simulate command-line arguments
sys.argv = [
    'script_name.py',  # Typically the script name, can be anything
    '--config', '../configs/waymo/voxelnet/two_stage/waymo_centerpoint_voxelnet_graphrcnn_6epoch_freeze.py',
    '--work_dir', '../work_dirs/waymo_centerpoint_voxelnet_graphrcnn_6epoch_freeze',
    '--checkpoint', '/home/rajeev-gupta/sensyn_ws/src/GraphRCNN/work_dirs/waymo_centerpoint_voxelnet_graphrcnn_6epoch_freeze/latest.pth',
    # '--txt_result', 'True',
    '--gpus', '1',
    '--local_rank', '0',
    # '--speed_test'
]

# Call the parse_args function
args = parse_args()

# Check the parsed arguments
print(args)

cfg = Config.fromfile(args.config)
cfg.local_rank = args.local_rank


Namespace(checkpoint='/home/rajeev-gupta/sensyn_ws/src/GraphRCNN/work_dirs/waymo_centerpoint_voxelnet_graphrcnn_6epoch_freeze/latest.pth', config='../configs/waymo/voxelnet/two_stage/waymo_centerpoint_voxelnet_graphrcnn_6epoch_freeze.py', gpus=1, launcher='none', local_rank=0, speed_test=False, testset=False, txt_result=False, work_dir='../work_dirs/waymo_centerpoint_voxelnet_graphrcnn_6epoch_freeze')


In [6]:

distributed = False if args.launcher == 'none' else True

if distributed:
    torch.cuda.set_device(args.local_rank)
    torch.distributed.init_process_group(backend="nccl", init_method="env://")

    cfg.gpus = torch.distributed.get_world_size()
else:
    cfg.gpus = args.gpus


In [7]:

# init logger before other steps
logger = get_root_logger(cfg.log_level)
logger.info("Distributed testing: {}".format(distributed))
logger.info(f"torch.backends.cudnn.benchmark: {torch.backends.cudnn.benchmark}")


2024-06-11 23:08:49,048 - INFO - Distributed testing: False
2024-06-11 23:08:49,048 - INFO - torch.backends.cudnn.benchmark: False


In [8]:

model = build_detector(cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)


2024-06-11 23:08:49,155 - INFO - Finish RPN Initialization
2024-06-11 23:08:49,156 - INFO - num_classes: [3]
2024-06-11 23:08:49,163 - INFO - Finish CenterHead Initialization


***************************model is build_from_cfg***************************
***************************model is build_from_cfg***************************
***************************model is build_from_cfg***************************
***************************model is build_from_cfg***************************
***************************model is build_from_cfg***************************
***************************model is build_from_cfg***************************
Use HM Bias:  -2.19
load_state_dict
The model and loaded state dict do not match exactly

unexpected key in source state_dict: single_det.backbone.conv_input.0.weight, single_det.backbone.conv_input.1.weight, single_det.backbone.conv_input.1.bias, single_det.backbone.conv_input.1.running_mean, single_det.backbone.conv_input.1.running_var, single_det.backbone.conv1.0.conv1.weight, single_det.backbone.conv1.0.conv1.bias, single_det.backbone.conv1.0.bn1.weight, single_det.backbone.conv1.0.bn1.bias, single_det.backbone.conv1.0.bn1

In [9]:

if args.testset:
    print("Use Test Set")
    dataset = build_dataset(cfg.data.test)
else:
    print("Use Val Set")
    dataset = build_dataset(cfg.data.val)


Use Val Set
Using 1 sweeps
Using 196 Frames


In [10]:
type(dataset)

det3d.datasets.waymo.waymo.WaymoDataset

In [11]:

data_loader = build_dataloader(
    dataset,
    batch_size=cfg.data.samples_per_gpu if not args.speed_test else 1,
    workers_per_gpu=cfg.data.workers_per_gpu,
    dist=distributed,
    shuffle=True,
)


In [12]:

# checkpoint = load_checkpoint(model, args.checkpoint, map_location="cpu")

# put model on gpus
if distributed:
    model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model)
    model = DistributedDataParallel(
        model.cuda(cfg.local_rank),
        device_ids=[cfg.local_rank],
        output_device=cfg.local_rank,
        # broadcast_buffers=False,
        find_unused_parameters=False,
    )
else:
    # model = fuse_bn_recursively(model)
    model = model.cuda()

model.eval()
mode = "val"


RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

In [13]:

logger.info(f"work dir: {args.work_dir}")
if cfg.local_rank == 0:
    prog_bar = torchie.ProgressBar(len(data_loader.dataset) // cfg.gpus)

detections = {}
cpu_device = torch.device("cpu")

start = time.time()

start = int(len(dataset) / 3)
end = int(len(dataset) * 2 /3)

time_start = 0 
time_end = 0 

print('\ndataloader type: ', type(data_loader))


2024-06-11 23:09:47,283 - INFO - work dir: ../work_dirs/waymo_centerpoint_voxelnet_graphrcnn_6epoch_freeze


[                              ] 0/196, elapsed: 0s, ETA:
dataloader type:  <class 'torch.utils.data.dataloader.DataLoader'>


In [14]:
dataset.__len__()

196

In [ ]:

for i, data_batch in enumerate(data_loader):
    if i == start:
        torch.cuda.synchronize()
        time_start = time.time()

    if i == end:
        torch.cuda.synchronize()
        time_end = time.time()

    with torch.no_grad():
        outputs = batch_processor(
            model, data_batch, train_mode=False, local_rank=args.local_rank,
        )
    for output in outputs:
        token = output["metadata"]["token"]
        for k, v in output.items():
            if k not in [
                "metadata",
            ]:
                output[k] = v.to(cpu_device)
        detections.update(
            {token: output,}
        )
        if args.local_rank == 0:
            prog_bar.update()

synchronize()


[]


ValueError: need at least one array to concatenate

In [ ]:

# all_predictions = all_gather(detections)

# print("\n Total time per frame: ", (time_end -  time_start) / (end - start))

# if args.local_rank != 0:
#     return

# predictions = {}
# for p in all_predictions:
#     predictions.update(p)

# if not os.path.exists(args.work_dir):
#     os.makedirs(args.work_dir)

# save_pred(predictions, args.work_dir)

# result_dict, _ = dataset.evaluation(copy.deepcopy(predictions), output_dir=args.work_dir, testset=args.testset)

# if result_dict is not None:
#     for k, v in result_dict["results"].items():
#         print(f"Evaluation {k}: {v}")

# if args.txt_result:
#     assert False, "No longer support kitti"
